<a href="https://colab.research.google.com/github/wei-jun7/DataAnalytics2023_WeijunLi/blob/main/hw1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print('hw1 begin: ')

hw1 begin: 
